Для итогового проекта будет использоваться датасет для определения фейковых новостей https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [2]:
df1 = pd.read_csv('True.csv')
df1.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [3]:
df1['target'] = 0

In [4]:
df1

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [5]:
df2 = pd.read_csv('Fake.csv')
df2.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
df2['target'] = 1

In [7]:
df2

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


Объединим данные с фековыми новостями и с настоящими. Затем перемещаем строки итогового датафрейма

In [8]:
df = pd.concat([df1, df2], ignore_index=True)
df

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [9]:
df = df.sample(frac=1)
df

,title,text,subject,date,target
16593,Iran sentences 'Mossad agent' to death over sc...,LONDON (Reuters) - Iran has sentenced to death...,worldnews,"October 24, 2017",0
2495,U.S. officials discussing withholding some aid...,WASHINGTON (Reuters) - U.S. officials are disc...,politicsNews,"July 26, 2017",0
29187,NRA Had This To Say After ANOTHER School Shoo...,The National Rifle Association (NRA) has been ...,News,"March 1, 2016",1
1359,NRA opposes outright U.S. ban on gun devices u...,WASHINGTON (Reuters) - The powerful U.S. gun l...,politicsNews,"October 8, 2017",0
22291,Eric Trump Gives DUMBEST Defense For His Brot...,The White House is getting really nervous toda...,News,"July 11, 2017",1
...,...,...,...,...,...
23558,Trump Screams ‘FAKE NEWS’ Over His Tax Return...,The White House may have verified the legitima...,News,"March 15, 2017",1
27172,Louie Gohmert Gets Completely Unhinged And Sc...,"Whenever big news happens in Washington, the n...",News,"June 23, 2016",1
31385,RINO SENATOR From “Red State” Tells Illegal Al...,Here are a few facts explaining who the DACA r...,politics,"Sep 6, 2017",1
632,Senate leaders look to work with White House t...,WASHINGTON (Reuters) - U.S. Senate Majority Le...,politicsNews,"November 14, 2017",0


In [10]:
df['target'].value_counts()

1    23481
0    21417
Name: target, dtype: int64

Разделим данные на train/test и сохраним выборку

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df, df['target'],
                                                    test_size=0.33, random_state=42)

X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)


X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [12]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [13]:
features = ['title', 'text', 'subject']
target = 'target'

In [ ]:
Соберём пайплайн

In [14]:
title = Pipeline([
                ('imputer', TextImputer('title', '')),
                ('selector', ColumnSelector(key='title')),
                ('tfidf', TfidfVectorizer())
            ])

text = Pipeline([
                ('imputer', TextImputer('text', '')),
                ('selector', ColumnSelector(key='text')),
                ('tfidf', TfidfVectorizer())
            ])

subject = Pipeline([
                ('imputer', TextImputer('subject', '')),
                ('selector', ColumnSelector(key='subject')),
                ('tfidf', TfidfVectorizer())
            ])


feats = FeatureUnion([('title', title),
                      ('text', text),
                      ('subject', subject)])

Дисбаланса классов нет, используем логистическую регрессию

In [15]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

Wall time: 17.8 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('title',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='title',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='text',
                                                                              value='')),
   

In [16]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('title',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='title',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='title')),
                                                  ('tfidf', TfidfVectorizer())])),
                                 ('text',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='text',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='text')),
                                                  ('tfidf', TfidfVectorizer())])),
                       

Сохраним наш пайплайн

In [17]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

Проверим работоспособность нашего пайплайна

In [18]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [19]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [20]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('title',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='title',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='text',
                                                                              value='')),
   

In [21]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [22]:
preds[:10]

array([1.36838696e-03, 1.01463448e-03, 9.98973535e-01, 1.00376345e-03,
       1.17383754e-03, 1.44065062e-03, 1.12105673e-03, 9.45772993e-04,
       1.16571584e-03, 1.29091675e-03])

In [23]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.992154666311357, F-Score=1.000, Precision=1.000, Recall=1.000


In [24]:
from flask import Flask, request, jsonify

Создадим сервис для обработки запросов к модели

In [25]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [26]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [27]:
app = Flask(__name__)


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    title, text, subject = "", "", ""
    request_json = request.get_json()
    
    if request_json["title"]:
        description = request_json['title']
    
    if request_json["text"]:
        company_profile = request_json['text']
                
    if request_json["subject"]:
        benefits = request_json['subject']
    
    print(title)  
    preds = model.predict_proba(pd.DataFrame({"title": [description],
                                              "text": [company_profile],
                                              "subject": [benefits]}))
    data["predictions"] = preds[:, 1][0]
    data["title"] = description
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Feb/2023 15:10:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2023 15:10:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Feb/2023 15:27:41] "POST //predict HTTP/1.1" 200 -



OK


Тестовый клиент. Возьмем свежую новость. Тестовый клиент в отдельном ноутбуке test_client